# Solve Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

You are encouraged to use AI assistants (like ChatGPT or Claude) and any IDE of your choice to develop your solution. Many modern IDEs (such as PyCharm, or Cursor) can work with Jupiter files directly.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [2]:
# Install the OpenAI Python package.
%pip install openai httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [15]:
# Code example of OpenAI communication

from openai import OpenAI
import os

# client = OpenAI(
#     # In order to use provided API key, make sure that models you create point to this custom base URL.
#     base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
#     # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
#     api_key='<OPENAI API KEY: Use one provided by Crossover or your own>'
# )

# completion = client.chat.completions.create(
#   model="gpt-4o",
#   messages=[
#     {"role": "user", "content": "Hello!"}
#   ]
# )

# print(completion.choices[0].message)

client = OpenAI(
 api_key="custom-API-KEY"
)


completion = client.chat.completions.create(
 model="gpt-4o",
 store=True,
 messages=[
   {"role": "user", "content": "write a haiku about ai"}
 ]
)


print(completion.choices[0].message)


ChatCompletionMessage(content='Silent circuits hum,  \nMachine dreaming in logic—  \nWisdom born of code.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [4]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


In [5]:
# Code example of generating output document

# Creates a new shared Google Worksheet every invocation with the proper structure
# Note: This code should be executed from the google colab once you are ready, it will not work locally
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# IMPORTANT: You need to authenticate the user to be able to create new worksheet
# Insert the authentication snippet from the official documentation to create a google client:
auth.authenticate_user()
creds, _ = default()

gc = gspread.authorize(creds)
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=qzi9VsEqzI-o

# This code goes after creating google client
output_document = gc.create('Solving Business Problems with AI - Output')

# Create 'email-classification' sheet
email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet.update([['email ID', 'category']], 'A1:B1')

# Example of writing the data into the sheet
# Assuming you have your classification in the email_classification_df DataFrame
# set_with_dataframe(email_classification_sheet, email_classification_df)
# Or directly update cells: https://docs.gspread.org/en/latest/user-guide.html#updating-cells

# Create 'order-status' sheet
order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')

# Create 'order-response' sheet
order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
order_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Create 'inquiry-response' sheet
inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Remove the default "Sheet1" (now that other sheets exist)
output_document.del_worksheet(output_document.sheet1)

# Share the spreadsheet publicly
output_document.share('', perm_type='anyone', role='reader')

# This is the solution output link, paste it into the submission form
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")

Shareable link: https://docs.google.com/spreadsheets/d/1prs-yNP647bfhe8c42YHoAqGHB4Y2jJUUlMOFby9BJ8


# Task 1. Classify emails

##Challenges:

#### 1. Ambiguity in Language:
- Emails can be unclear about whether they are inquiries or orders. Customers may use different phrases, sentence structures, or levels of detail, making it difficult to define strict rules for classification.

#### 2. Vague References:
- Customers may not give exact product details, making it hard to know if they intend to order.

#### 3. Mixed Intent:
- Emails might have both questions and orders, making it tricky to determine the primary intent.

##Ideation:

#### 1. Leverage LLMs:
- Use the power of large language models (LLMs) like GPT-4 to understand the nuances of human language and interpret the intent behind emails.

#### 2. Focus on Intent:
- Concentrate on the overall intent of the email rather than individual phrases or keywords.

#### 3. Iterative Refinement:
- Start with a basic prompt and iteratively refine it based on the LLM's performance on various email examples.

##Approach:

#### 1. Detailed Prompt Engineering:
- Carefully design the prompt for the LLM to provide clear instructions and examples for classifying emails.
- Include precise definitions of "product inquiry" and "order request" with illustrative examples.
- Provide guidelines for handling vague product references and prioritizing explicit order intent.

#### 2. Validation of LLM Output:
- Add a validation step to ensure the LLM's response is always one of the expected categories ("product inquiry" or "order request").

#### 3. Testing and Refinement:
- Test the classification with a variety of email examples and refine the prompt based on the LLM's performance to improve accuracy and handle edge cases.

* Batch processing and streaming are not used due to the following reasons:

   * Limited number of emails and a single prompt: The small dataset offers minimal performance benefits from batching or streaming.

   * Individual email responses: Each email requires a unique, personalized reply, best handled individually.

   * Stock management: Immediate stock updates ensure accuracy and prevent conflicts, which batching or streaming could complicate.

   * API token limitations: Constraints make both batch processing and streaming impractical.

   * Edge Case Handling: Addressing ambiguous or mixed-intent emails is easier when processed individually rather than in batches.  
   
Note: Batch processing or streaming could be reconsidered if the dataset grows or API constraints change.

In [6]:
import time

# Initialize an empty DataFrame to store classification results
email_classification_df = pd.DataFrame(columns=['email ID', 'category'])

# Iterate through each email in the emails_df DataFrame
for index, email in emails_df.iterrows():
    try:
        # Construct the prompt for the LLM
        prompt = f"""
You are an AI assistant helping a fashion store categorize customer emails.

Here's an email from a customer:
Subject: {email['subject']}
Body: {email['message']}

Classify this email as either:

1. "product inquiry" - If the email contains questions or expresses general interest in products,
                       OR if the customer mentions an intent to purchase BUT does NOT provide
                       specific details about the product(s) they want (e.g., product name, ID,
                       or a very clear description that uniquely identifies the product).
                       Examples of vague descriptions: "the one with...", "those amazing...", "from your latest collection..."

2. "order request" - If the email explicitly states a desire to buy AND provides clear details
                      about the product(s) they want, even if quantities are not specified.

The output should only be one of these categories:
* product inquiry
* order request
"""

        # Call the OpenAI ChatCompletion API to get the classification
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )

        # Extract the category from the LLM's response
        category = completion.choices[0].message.content.strip().lower()  # Normalize to lowercase

        # Validate the LLM response
        valid_categories = ["product inquiry", "order request"]
        if category not in valid_categories:
            print(f"Unexpected category for email ID {email['email_id']}: {category}")
            continue

        # Append to the results DataFrame
        email_classification_df.loc[len(email_classification_df)] = [email['email_id'], category]

        # Log progress
        print(f"Processed email ID: {email['email_id']} - Category: {category}")

        # Optional: Add a delay to avoid hitting rate limits
        time.sleep(0.5)  # Adjust the sleep duration if needed

    except Exception as e:
        print(f"Error processing email ID {email['email_id']}: {e}")
        continue

# Write the classification results to the Google Sheet
set_with_dataframe(email_classification_sheet, email_classification_df)

print("Classification results uploaded successfully!")

Processed email ID: E001 - Category: order request
Processed email ID: E002 - Category: order request
Processed email ID: E003 - Category: product inquiry
Processed email ID: E004 - Category: order request
Processed email ID: E005 - Category: product inquiry
Processed email ID: E006 - Category: product inquiry
Processed email ID: E007 - Category: order request
Processed email ID: E008 - Category: order request
Processed email ID: E009 - Category: product inquiry
Processed email ID: E010 - Category: order request
Processed email ID: E011 - Category: product inquiry
Processed email ID: E012 - Category: product inquiry
Processed email ID: E013 - Category: order request
Processed email ID: E014 - Category: order request
Processed email ID: E015 - Category: product inquiry
Processed email ID: E016 - Category: product inquiry
Processed email ID: E017 - Category: product inquiry
Processed email ID: E018 - Category: order request
Processed email ID: E019 - Category: order request
Processed ema

# Task 2. Process order requests

##Challenges:

#### 1. Product ID Extraction:
- Accurately extracting product IDs from unstructured email text can be challenging due to varying formats, abbreviations, or typos. Some emails may only mention product names, requiring mapping to the correct product ID.

#### 2. Quantity Extraction:
- Accurately determining the quantity of each product ordered, especially when customers use phrases like "all remaining" or ranges ("3-4").
Handling cases where the quantity is implicitly mentioned or requires contextual understanding.

#### 3. Dynamic Stock Updates:
- Ensuring stock levels are updated correctly and immediately after each order to prevent overselling and maintain accurate inventory.

####4. Informative Email Responses:
- Crafting clear, personalized emails to customers about their orders, including confirmations, out-of-stock situations, and alternative suggestions.

##Ideation:

#### 1. Leverage LLMs for Extraction:
- Utilize LLMs to extract product IDs, names, and quantities, minimizing the need for manual regex-based extraction.

####2. LLM-Driven Stock Management:
- Delegate stock checking and updating to the LLM, reducing the need for separate stock processing functions.

####3. Contextual Understanding:
- Train the LLM to understand the context of emails, including identifying order intent, handling special quantity cases, and suggesting relevant alternatives.

##Approach:

#### 1. LLM-Based Extraction:
- The extract_products_with_llm function uses an LLM to extract product IDs, names, and quantities from emails.
The prompt includes comprehensive instructions and rules to guide the LLM in handling various scenarios and edge cases.

#### 2. Stock Validation and Update:
- The check_stock_and_update_status function validates the extracted order against the available stock.
It handles special quantity cases like "all remaining" and updates the stock levels accordingly.

#### 3. Email Response Generation:
- The generate_order_response_email function uses an LLM to generate personalized email responses to customers.
The prompt includes the order summary and instructions for handling different order statuses and suggesting alternatives.

#### 4. Simplified Customer Interaction:
- The customer name extraction was removed to simplify the process and ensure consistent, professional responses.
All emails use "Dear Customer" as the greeting, avoiding potential errors and privacy concerns.

#### 5. Google Sheets Integration:
- The process_order_requests function orchestrates the entire process, including extracting orders, validating stock, generating responses, and saving the results to Google Sheets.

In [16]:
import ast
import re
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def extract_products_with_llm(email_subject, email_body, products_df):
    # Create a dictionary mapping product IDs to stock levels
    stock_dict = products_df.set_index("product_id")["stock"].to_dict()

    # Prepare the product catalog with stock information
    product_catalog_with_stock = "\n".join(
        [f"{row['product_id']} - {row['name']} (Stock: {row['stock']})"
         for _, row in products_df.iterrows()]
    )

    """
    Extracts product information (IDs, names, quantities) from an email using LLM.

    Parameters:
        email_subject (str): Subject of the email.
        email_body (str): Body of the email.
        products_df (pd.DataFrame): Product catalog DataFrame.

    Returns:
        list: Extracted product details as a list of dictionaries.
    """
    prompt = f"""
    You are an AI assistant helping a fashion store process customer emails.
    Extract the product IDs, names, and quantities from the following email:

    Subject: {email_subject}
    Body: {email_body}

    Product Catalog:
    {products_df[['product_id', 'name']].to_string(index=False)}

    ### Rules and Guidelines ###
    1. **Product Identification**:
       - Extract product IDs directly if mentioned (e.g., "LT2345", "CBT 89 01").
       - If product IDs are not mentioned, identify product names from the catalog and match them to their IDs.
       - Ignore gender-specific terms in product names (e.g., "Men's", "Women's").
       - Consider product descriptions or styles to infer matches (e.g., "sleek wallet" matches "Sleek Wallet").

    2. **Quantity Extraction**:
       - Use explicit quantities if mentioned (e.g., "2 pieces of LT2345").
       - For ranges (e.g., "3-4"), use the higher value (4).
       - For "all remaining," use the entire available stock in the catalog.
       - Default to 1 if quantity is not mentioned but the order intent is clear.

    3. **Contextual Rules**:
       - Exclude references to:
         - **Past purchases**: "I bought", "I've purchased", "I've had".
         - **Future intentions**: "next time", "planning to", "will order".
         - **General appreciation**: "I love the quality of...".
       - Include only products where the customer expresses **current intent to order** (e.g., "I would like to order", "please send", "I need").

    4. **Output Format**:
       Return the results as a Python list of dictionaries:
       [
           {{"product_id": "XXX0000", "name": "Product Name", "quantity": 2}},
           ...
       ]
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        raw_output = response.choices[0].message.content.strip()

        # Print raw response for debugging
        #print(f"Raw LLM Response: {raw_output}")

        # Extract JSON-like content using regex
        match = re.search(r"\[.*?\]", raw_output, re.DOTALL)
        if match:
            json_like_content = match.group(0)
            extracted_products = ast.literal_eval(json_like_content)
            return extracted_products
        else:
            print("No valid JSON-like content found in the response.")
            return []

    except (SyntaxError, ValueError) as parse_error:
        print(f"Parsing Error: {parse_error}")
        return []
    except Exception as e:
        print(f"Error during LLM extraction: {e}")
        return []


# Function to validate stock and update statuses
def check_stock_and_update_status(order_status_df, products_df):
    stock_dict = products_df.set_index("product_id")["stock"].to_dict()
    partial_orders = {}

    for index, row in order_status_df.iterrows():
        product_id = row["product ID"]
        quantity = row["quantity"]

        try:
            # Get the available stock for the product
            available_stock = stock_dict.get(product_id, 0)

            # Handle "all remaining" explicitly
            if isinstance(quantity, str) and quantity.lower() == "all remaining":
                quantity = available_stock  # Replace "all remaining" with stock value
            elif isinstance(quantity, str):
                print(f"Invalid quantity format for product {product_id}: {quantity}")
                quantity = 1  # Default to 1 if unparseable

            # Ensure quantity is an integer
            quantity = int(quantity)

            if quantity == 0:
                print(f"Excluding product {product_id} with quantity = 0.")
                continue

            # Update statuses
            if quantity > available_stock:
                row["status"] = "out of stock"
                row["quantity"] = available_stock  # Update quantity to available stock
                partial_orders[product_id] = {
                    "email_id": row["email ID"],
                    "requested": quantity,
                    "available": available_stock,
                }
            else:
                row["status"] = "created"
                stock_dict[product_id] -= quantity  # Deduct stock

            row["quantity"] = quantity  # Keep the processed quantity
            order_status_df.loc[index] = row  # Update the row in DataFrame

        except Exception as e:
            print(f"Error processing product {product_id}: {e}")
            continue

    return order_status_df, partial_orders


def generate_order_response_email(email_id, order_status_df, products_df, partial_orders):
    """
    Generates an email response for an order using the LLM,
    including information about partial shipments.

    Parameters:
        email_id (str): The email ID of the customer.
        order_status_df (pd.DataFrame): DataFrame containing order details.
        products_df (pd.DataFrame): DataFrame containing product catalog.
        partial_orders (dict): Dictionary with details of partially fulfilled orders.

    Returns:
        str: The generated email response as a string.
    """
    # Filter order items for the current email ID
    order_items = order_status_df[order_status_df['email ID'] == email_id]

    # Create an informative order summary for the LLM
    order_summary = ""
    for _, row in order_items.iterrows():
        product_name = products_df.loc[products_df['product_id'] == row['product ID'], 'name']
        if product_name.empty:
            product_name = "Unknown Product"
        else:
            product_name = product_name.iloc[0]

        order_summary += f"""
        Product: {product_name} (Product ID: {row['product ID']})
        Quantity Ordered: {row['quantity']}
        Status: {row['status']}
        """

        # Add partial order details to the summary
        if row['product ID'] in partial_orders and partial_orders[row['product ID']]['email_id'] == email_id:
            available_qty = partial_orders[row['product ID']]['available']
            order_summary += f"Available Quantity: {available_qty}\n"

    # Compose the prompt for the LLM
    prompt = f"""
    You are an AI assistant helping a fashion store with order fulfillment.

    Compose an email to the customer about their order (email ID: {email_id}).

    Order Summary:
    {order_summary}

    Instructions:
    * Start with a professional greeting.
    * Accurately describe the status of each item in the order.
    * If an item is 'out of stock' and has an 'Available Quantity' listed, it means it's partially fulfilled.
      Clearly state that only the available quantity can be shipped now and offer the following options:
        * Option 1: Ship available items now and the remaining quantity when it's back in stock.
        * Option 2: Wait and ship the entire order together when all items are back in stock.
    * If an item is 'out of stock' and does NOT have an 'Available Quantity' listed, it means it's completely out of stock.
      Inform the customer and suggest waiting for restock.
    * If an item has the status 'created', confirm that it is ready to ship.
    * If applicable, suggest similar items from the same category as the ordered items.
    * End with a professional closing.
    * Write in a professional and helpful tone, ready to send to the customer.
    """

    try:
        # Call the LLM to generate the response
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return completion.choices[0].message.content.strip()

    except Exception as e:
        print(f"Error generating email response for {email_id}: {e}")
        return None


def process_order_requests(email_classification_df, emails_df, products_df, order_status_sheet, order_response_sheet):
    """
    Processes order requests, updates stock, generates customer responses,
    and saves results to Google Sheets.

    Returns:
        pd.DataFrame: The processed order_status_df.
    """
    order_requests_df = email_classification_df[email_classification_df['category'] == 'order request']
    print(f"Processing {len(order_requests_df)} order requests.")

    order_status_df = pd.DataFrame(columns=['email ID', 'product ID', 'quantity', 'status'])

    for _, email in order_requests_df.iterrows():
        email_id = email['email ID']
        subject = emails_df.loc[emails_df['email_id'] == email_id, 'subject'].values[0]
        body = emails_df.loc[emails_df['email_id'] == email_id, 'message'].values[0]

        extracted_products = extract_products_with_llm(subject, body, products_df)
        print(f"Extracted Products for Email ID {email_id}:\n{extracted_products}")

        for product in extracted_products:
            order_status_df = pd.concat([
                order_status_df,
                pd.DataFrame({'email ID': [email_id], 'product ID': [product['product_id']],
                              'quantity': [product['quantity']], 'status': ["pending"]})
            ], ignore_index=True)

    # Validate stock and update statuses
    order_status_df, partial_orders = check_stock_and_update_status(order_status_df, products_df)

    # Exclude rows with "pending" status
    order_status_df = order_status_df[order_status_df["status"] != "pending"]

    # Convert DataFrame columns to Python-native types
    column_types = {
        "email ID": "str",
        "product ID": "str",
        "quantity": "int",
        "status": "str",
    }
    order_status_df = order_status_df.astype(column_types)

    # Sort and save order_status_df to Google Sheets
    order_status_df = order_status_df.sort_values(by='email ID')
    set_with_dataframe(order_status_sheet, order_status_df)
    print("Order status uploaded successfully!")

    # Generate email responses
    order_response_df = pd.DataFrame(columns=['email ID', 'response'])
    for email_id in order_status_df['email ID'].unique():
        response_email = generate_order_response_email(email_id, order_status_df, products_df, partial_orders)
        if response_email:
            print(f"Generated Response for Email ID {email_id}:\n{response_email}\n")
            order_response_df = pd.concat([
                order_response_df,
                pd.DataFrame({'email ID': [email_id], 'response': [response_email]})
            ], ignore_index=True)

    # Save responses to Google Sheets
    set_with_dataframe(order_response_sheet, order_response_df)
    print("Order responses uploaded successfully!")

    return order_status_df

# Main Execution
# --- Call the processing function ---
order_status_df = process_order_requests(email_classification_df, emails_df, products_df, order_status_sheet, order_response_sheet)

# --- Print the order_status_df to verify ---
print("\nOrder Status DataFrame:")
print(order_status_df.to_markdown(index=False, numalign="left", stralign="left"))  # Print in markdown format

# --- Update Google Sheets ---
set_with_dataframe(order_status_sheet, order_status_df)
print("Order status and responses uploaded successfully!")


Processing 11 order requests.
Extracted Products for Email ID E001:
[{'product_id': 'LTH0976', 'name': 'Leather Bifold Wallet', 'quantity': 'all remaining'}]
Extracted Products for Email ID E002:
[{'product_id': 'VBT2345', 'name': 'Vibrant Tote', 'quantity': 1}]
Extracted Products for Email ID E004:
[{'product_id': 'SFT1098', 'name': 'Infinity Scarf', 'quantity': 4}]
Extracted Products for Email ID E007:
[{'product_id': 'CLF2109', 'name': 'Cable Knit Beanie', 'quantity': 5}, {'product_id': 'FZZ1098', 'name': 'Fuzzy Slippers', 'quantity': 2}]
Extracted Products for Email ID E008:
[{'product_id': 'VSC6789', 'name': 'Versatile Scarf', 'quantity': 1}]
Extracted Products for Email ID E010:
[{'product_id': 'RSG8901', 'name': 'Retro Sunglasses', 'quantity': 1}]
Extracted Products for Email ID E013:
[{'product_id': 'SLD7654', 'name': 'Slide Sandals', 'quantity': 1}]
Extracted Products for Email ID E014:
[{'product_id': 'SWL2345', 'name': 'Sleek Wallet', 'quantity': 1}]
Extracted Products for E

# Task 3. Handle product inquiry

##Challenges:

#### 1. LLM Token Limits:

- LLMs have token limits, restricting the amount of text they can process. With a large product catalog (over 100,000 products), it's infeasible to include all product details in the prompt.

#### 2. Efficient Information Retrieval:

- Quickly and accurately retrieving relevant product information from the catalog based on the customer's inquiry.

#### 3. Concise and Informative Responses:

- Generating responses that are concise, informative, and address the customer's query effectively, even with limited context.

##Ideation:

#### 1. Retrieval-Augmented Generation (RAG):

- Use RAG to retrieve relevant product information from the catalog based on the customer's inquiry, allowing the LLM to focus on a smaller, more manageable context.

#### 2. Vector Database:

- Utilize a vector database to store and efficiently search product information based on semantic similarity.

##Approach:

#### 1. Vector Store:

- Create a vector store using FAISS (Facebook AI Similarity Search).
- Embed product descriptions using OpenAIEmbeddings and store them in the vector store along with product metadata.

####2. Retrieval:

- Retrieve the most relevant products from the vector store based on the customer's inquiry.
- Use LangChain's RetrievalQA chain to manage the retrieval process.

####3. LLM with Context:

- Provide the LLM with the customer's inquiry and the retrieved product information to generate a concise and informative response.
- Use a prompt template to structure the LLM's input and guide its response generation.

####4. Concise Responses:

- Set a max_tokens limit for the LLM to ensure concise responses.
- Instruct the LLM in the prompt to avoid unnecessary details and focus on specific answers.

####5. Google Sheets Integration:

- Save the generated responses to a Google Sheet using the set_with_dataframe function.

In [8]:
# NOTE: When testing this in a Jupyter notebook on Colab, be sure to switch the Runtime type to T4-GPU or any other hardware accelerator (if you're feeling *premium*) 😉

!pip install langchain-community
!pip install tiktoken
!pip install faiss-gpu

import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

def handle_product_inquiry(email_classification_df, emails_df, products_df, inquiry_response_sheet):
    """
    Handles product inquiries using Retrieval-Augmented Generation (RAG) and a vector store.
    Populates the inquiry-response sheet with concise responses.
    """
    # --- LLM and Vector Store Initialization ---
    os.environ["OPENAI_API_KEY"] = "<API-KEY>"
    llm = OpenAI(temperature=0, max_tokens=100, openai_api_key=os.environ["OPENAI_API_KEY"])

    embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
    texts = products_df["description"].tolist()
    metadatas = products_df.drop(columns=["description"]).to_dict("records")
    vector_store = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

    # --- Prompt Setup ---
    prompt_template = """Use the following context to answer the customer's query:
    Context: {context}
    Question: {question}
    Instructions:
    - Start with a professional greeting.
    - Provide concise, specific answers to the query using the context and product catalog.
    - Avoid including unnecessary details.
    - End professionally.
    Answer:"""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(),
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    # --- Initialize Response DataFrame ---
    inquiry_response_df = pd.DataFrame(columns=['email ID', 'response'])

    # --- Filter Product Inquiry Emails ---
    product_inquiry_email_ids = email_classification_df[email_classification_df['category'] == 'product inquiry']['email ID'].tolist()
    product_inquiry_emails_df = emails_df[emails_df['email_id'].isin(product_inquiry_email_ids)]

    # --- Process Each Inquiry ---
    for index, row in product_inquiry_emails_df.iterrows():
        email_id = row['email_id']
        email_subject = row['subject']
        email_body = row['message']

        query = f"Subject: {email_subject}\nBody: {email_body}"
        #print(f"Processing Email ID: {email_id}\nQuery: {query}")

        try:
            # Generate Response
            response = qa_chain({"query": query})
            answer = response['result']

            # Append Response to DataFrame
            inquiry_response_df = pd.concat([
                inquiry_response_df,
                pd.DataFrame({'email ID': [email_id], 'response': [answer]})
            ], ignore_index=True)

            print(f"Processed Response for Email ID: {email_id}\nResponse: {answer}\n")

        except Exception as e:
            print(f"Error processing email ID {email_id}: {e}")
            continue

    # --- Update Google Sheets ---
    set_with_dataframe(inquiry_response_sheet, inquiry_response_df)
    print("Inquiry responses uploaded successfully!")
    return inquiry_response_df


# Main Execution
# --- Call the handle_product_inquiry function ---
inquiry_response_df = handle_product_inquiry(email_classification_df, emails_df, products_df, inquiry_response_sheet)

# --- Print the inquiry_response_df to verify ---
print("\nInquiry Response DataFrame:")
print(inquiry_response_df.to_markdown(index=False, numalign="left", stralign="left"))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.3 MB/s eta 0:00:00
Processing Email ID: E003
Query: Subject: Need your help
Body: Hello, I need a new bag to carry my laptop and documents for work. My name is David and I'm having a hard time deciding which would be better - the LTH1098 Leather Backpack or the Leather Tote? Does one have more organizational pockets than the other? Any insight would be appreciated!


<ipython-input-8-92e6b11c2bbd>:70: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"query": query})


Processed Response for Email ID: E003
Response:  Hello David, thank you for reaching out to us. Our LTH1098 Leather Backpack and Leather Tote are both great options for carrying your laptop and documents for work. The backpack features multiple compartments and a padded laptop sleeve for organization, while the tote has a spacious interior and multiple pockets for storage. Both are crafted from premium leather and are perfect for work, travel, or everyday use. I hope this helps with your decision. Let us know if you have any further questions. Thank you.

Processing Email ID: E005
Query: Subject: Inquiry on Cozy Shawl Details
Body: Good day, For the CSH1098 Cozy Shawl, the description mentions it can be worn as a lightweight blanket. At $22, is the material good enough quality to use as a lap blanket? Or is it more like a thick wrapping scarf? I'm considering buying it as a gift for my grandmother. Thank you!
Processed Response for Email ID: E005
Response:  Hello, thank you for your in

##**Overall Approach**

#### 1. Thorough Understanding:
- I began by carefully analyzing the task requirements, paying close attention to the nuances of quantity extraction, stock management, and email generation. This ensured a clear understanding of the desired outcome and the constraints involved.

#### 2. Strategic Planning:
- I devised a strategic plan, breaking down the complex task into manageable components. This involved identifying the key steps in the email processing pipeline, including product ID extraction, quantity extraction, stock checking, and response generation.

#### 3. Iterative Refinement:
- I adopted an iterative approach, continuously refining the code based on testing, analysis of the output, and identification of edge cases. This allowed for incremental improvements and ensured the solution's accuracy and reliability.

\\

**٩(◕‿◕｡)۶ This assessment was an engaging exercise where I tackled challenges in natural language processing, data extraction, and automated email response generation. Using a systematic and iterative approach, I focused on delivering accurate, efficient, and robust solutions while thoroughly enjoying the process. (✿◠‿◠)**

\\